In [1]:
import pandas as pd
import psycopg2
import user_credentials

In [2]:
# Connect to the climbing_db database
database_name = "climbing_db"
try:
    conn = psycopg2.connect(database = database_name, user = user_credentials.username, password = user_credentials.password, 
                            host = "localhost", port = "5432")
    cursor = conn.cursor()
except psycopg2.errors.OperationalError:
    print("Database connection not successful")

In [3]:
# Query routes data
cursor.execute('''
SELECT c.country, r.crag, r.sector, r.name, r.grade_mean, r.rating_tot, r.style
FROM countries AS c
JOIN routes AS r ON c.country_id = r.country
''')
routes_data = cursor.fetchall()

# Extract the data into a dataframe
routes_df = pd.DataFrame(routes_data, columns=['country', 'crag', 'sector', 'route name', 'grade', 'rating', 'style'])

In [4]:
# Query grades data
cursor.execute('SELECT * FROM grades')
grades_data = cursor.fetchall()

# Extract the data into a dataframe
grades_df = pd.DataFrame(grades_data, columns=['id', 'french', 'yds', 'v'])
grades_df.set_index('id', inplace=True)

In [5]:
cursor.close()
conn.close()

In [6]:
# Get user input
country = 'France'
grade_lower = 'V5'
grade_upper = 'V7'
style = 'Neutral'

In [7]:
# Choose desired grading scale
if not isinstance(grade_lower, int):
    grade_scales = ['french', 'yds', 'v']
    selected_scale = ''
    if '.' in grade_lower:
        selected_scale = 'yds'
    elif 'v' in grade_lower.lower():
         selected_scale = 'v'
    else:
         selected_scale = 'french'
    grade_scales.remove(selected_scale)
    grades_df.drop(grade_scales, axis=1, inplace=True)
    df = pd.merge(routes_df, grades_df, left_on='grade', right_on='id')

In [15]:
# Filter for country and style
df = df[df['country'] == country]
df = df[df['style'] == style]
df

,country,crag,sector,route name,grade,rating,style,v
1640,France,Ailefroide,Gorge,Secteur F,49,-0.164902,Neutral,V3
1641,France,Alliat,Grotte Aux Fees,Les Fees Papillons,49,0.074479,Neutral,V3
1643,France,Alliat,Petite Grotte,Ecart Type,49,0.074479,Neutral,V3
1645,France,Ardeche,Les Actinidias,Tout Leffet Dune Petite Fee Variante,49,0.000701,Neutral,V3
1646,France,Ardeche,Les Branches,Le Roi De Lolympe,49,-0.164902,Neutral,V3
...,...,...,...,...,...,...,...,...
55823,France,Gorges Du Loup,Deverse,Kinematix,75,-0.081832,Neutral,V14
55824,France,Gorges Du Loup,Deverse,Puntx,75,-0.117354,Neutral,V14
55833,France,Pierrot Beach,Finales,Nice To Eat You,73,-0.041565,Neutral,V13
55843,France,Gorges Du Loup,Deverse,Abyss,74,-0.160239,Neutral,V13


In [40]:
# Filter for grade range
min_grade = grades_df[grades_df[selected_scale] == grade_lower].index.min()
max_grade = grades_df[grades_df[selected_scale] == grade_upper].index.max()
df = df[(df['grade'] >= min_grade) & (df['grade'] <= max_grade)]
df

,country,crag,sector,route name,grade,rating,style,v
13996,France,Ailefroide,Face Bouc,Elichat Botte,53,-0.025044,Neutral,V5
13997,France,Alliat,Grotte Aux Fees,Legende A Momo,53,-0.093088,Neutral,V5
14000,France,Alliat,Le Livre,Preface,53,-0.045211,Neutral,V5
14002,France,Ardeche,Les Actinidias,Les Cryptos Du Plein Air,53,-0.159781,Neutral,V5
14003,France,Balme Etrange,Gauche,Sans Toit Ni Loi,53,0.014634,Neutral,V5
...,...,...,...,...,...,...,...,...
54963,France,St Leger,Baleine,Artisanat Local,60,-0.043503,Neutral,V7
54967,France,St Leger,Praniania,Les Lectures Danne Alphabete,60,0.073419,Neutral,V7
54968,France,Tamee,Plage,Lentorse,60,-0.045211,Neutral,V7
54969,France,Turbie,Big Ben,Pestilence,60,-0.060611,Neutral,V7


In [46]:
# Keep crags with at least X routes
df.groupby('crag').count() > 5

,country,sector,route name,grade,rating,style,v
crag,,,,,,,
Ailefroide,True,True,True,True,True,True,True
Alliat,True,True,True,True,True,True,True
Ardeche,True,True,True,True,True,True,True
Balme,False,False,False,False,False,False,False
Balme Etrange,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...
Turbie,True,True,True,True,True,True,True
Venasque,True,True,True,True,True,True,True
Verdon,True,True,True,True,True,True,True
